In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm as tqdm, trange
from dataclasses import dataclass, field
from typing import Tuple, List, Any
from functools import partial
import pickle
import torch
from torch import nn
import torch.nn.functional as F

%matplotlib inline
sns.set_style("whitegrid")
sns.set_palette("colorblind")
plt.rcParams.update({'font.size': 14})

trange = partial(trange, position=0)
tqdm = partial(tqdm, position=0)

# Часть третья: расширим и углубим поиск
Крестики-нолики -- это, конечно, далеко не го, и обычный альфа-бета поиск с отсечением
здесь наверняка может работать идеально вплоть до довольно больших досок. Однако мы
всё-таки для этого учебного задания будем реализовывать более практически
релевантный метод MCTS -- заодно фактически получится и упражнение на многоруких
бандитов.

## Пункт 5
Реализуйте rollouts со случайной стратегией и (опционально) rollouts с
неслучайной, но простой стратегией (например, основанной на дополнении
нескольких паттернов или на Q-функции, которая у вас получилась в первом
пункте).

In [2]:
from tic_tac_toe import TicTacToe
from contracts import State, Action, Strategy, RandomStrategy, TicTacToeWrapper
from dqn import TicTacToeDQNWrapper, DQNStrategy
from mcts import rollout, RolloutStrategy, MCTSNode, MCTSStrategy

In [3]:
env = TicTacToeDQNWrapper(TicTacToe(3,3,3))

In [4]:
def show_test_game(env: TicTacToeWrapper,
                    crosses_strategy: Strategy,
                    naughts_strategy: Strategy,
                    show_only_final: bool = False):
    '''Играем тестовую партию между стратегиями, печатаем ход игры'''
    state, actions = env.reset()
    while True:
        if env.env.curTurn == 1:
            action = crosses_strategy.get_action(state, actions)
        else:
            action = naughts_strategy.get_action(state, actions)
        state, actions, crosses_reward, done = env.step(action)
        if not show_only_final:
            env.env.printBoard()
            print()
        if crosses_reward == 1:
            if show_only_final:
                env.env.printBoard()
            print("Крестики выиграли!")
            return
        if crosses_reward == -1:
            if show_only_final:
                env.env.printBoard()
            print("Нолики выиграли!")
            return
        if done:
            if show_only_final:
                env.env.printBoard()
            print("Ничья!")
            return

In [5]:
def run_episode(env: TicTacToeWrapper,
                        crosses_strategy: Strategy,
                        naughts_strategy: Strategy) -> int:
    """Генерирует эпизод по двум стратегиям, возвращает финальный reward для крестиков"""
    state, actions = env.reset()
    while True:
        if env.env.curTurn == 1:
            action = crosses_strategy.get_action(state, actions)
        else:
            action = naughts_strategy.get_action(state, actions)
        state, actions, crosses_reward, done = env.step(action)
        if done:
            return crosses_reward

In [6]:
@dataclass
class EvaluationResults:
    crosses_mean_reward: float
    naughts_mean_reward: float
    crosses_win_fraction: float
    naughts_win_fraction: float
    tie_fraction: float

In [7]:
def evaluate_strategy(env: TicTacToeWrapper,
                        crosses_strategy: Strategy,
                        naughts_strategy: Strategy,
                        num_iterations: int) -> EvaluationResults:
    """Генерирует num_iterations запусков по двум стратегиям
    и возвращает итоговые результаты"""
    crosses_rewards = [run_episode(env, crosses_strategy, naughts_strategy)
                                                for _ in trange(num_iterations)]
    crosses_rewards = np.array(crosses_rewards)
    crosses_mean_reward = crosses_rewards.mean()
    naughts_mean_reward = -crosses_mean_reward
    crosses_win_fraction = (crosses_rewards == 1).mean()
    naughts_win_fraction = (crosses_rewards == -1).mean()
    tie_fraction = (crosses_rewards == 0).mean()
    return EvaluationResults(crosses_mean_reward, naughts_mean_reward,
                             crosses_win_fraction, naughts_win_fraction, tie_fraction)

In [8]:
def save(obj: Any, path: str):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)


def load(path) -> Any:
    with open(path, 'rb') as f:
        return pickle.load(f)

In [9]:
random_strategy = RandomStrategy()

In [10]:
show_test_game(env, random_strategy, random_strategy, True)

---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x | x | o | 
---- ---- ---- -
| o | x | o | 
---- ---- ---- -
Крестики выиграли!


## Пробуем rollouts со случайной стратегией

In [11]:
rollout_stategy = RolloutStrategy(env, n_iter=10000, n_jobs=16, base_strategy=RandomStrategy())

Сыграем против случайной стратегии

In [12]:
show_test_game(env, rollout_stategy, random_strategy)

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
| o |   |   | 
---- ---- ---- -

---- ---- ---- -
| x |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
| o |   |   | 
---- ---- ---- -

---- ---- ---- -
| x |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
| o |   | o | 
---- ---- ---- -

---- ---- ---- -
| x |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
| o | x | o | 
---- ---- ---- -

---- ---- ---- -
| x |   |   | 
---- ---- ---- -
|   | x | o | 
---- ---- ---- -
| o | x | o | 
---- ---- ---- -

---- ---- ---- -
| x | x |   | 
---- ---- ---- -
|   | x | o | 
---- ---- ---- -
| o | x | o | 
---- ---- ---- -

Крестики выиграли!


In [13]:
show_test_game(env, random_strategy, rollout_stategy)

---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   | o |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   | o |   | 
---- ---- ---- -
|   |   | x | 
---- ---- ---- -

---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   | o |   | 
---- ---- ---- -
| o |   | x | 
---- ---- ---- -

---- ---- ---- -
|   | x | x | 
---- ---- ---- -
|   | o |   | 
---- ---- ---- -
| o |   | x | 
---- ---- ---- -

---- ---- ---- -
|   | x | x | 
---- ---- ---- -
|   | o | o | 
---- ---- ---- -
| o |   | x | 
---- ---- ---- -

---- ---- ---- -
| x | x | x | 
---- ---- ---- -
|   | o | o | 
---- ---- ---- -
| o |   | x | 
---- ---- ---- -

Крестики выиграли!


Против себя

In [14]:
show_test_game(env, rollout_stategy, rollout_stategy)

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
| o |   |   | 
---- ---- ---- -

---- ---- ---- -
| x |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
| o |   |   | 
---- ---- ---- -

---- ---- ---- -
| x |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
| o |   | o | 
---- ---- ---- -

---- ---- ---- -
| x |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
| o | x | o | 
---- ---- ---- -

---- ---- ---- -
| x |   |   | 
---- ---- ---- -
|   | x | o | 
---- ---- ---- -
| o | x | o | 
---- ---- ---- -

---- ---- ---- -
| x | x |   | 
---- ---- ---- -
|   | x | o | 
---- ---- ---- -
| o | x | o | 
---- ---- ---- -

Крестики выиграли!


Против стратегии, обученной в первой части

In [15]:
self_play_q_3x3 = load('models/self_play_q_3x3.pkl')

In [16]:
from qlearning import EpsilonGreedyStrategy

In [17]:
q_3x3_strategy = EpsilonGreedyStrategy(0, self_play_q_3x3)

In [18]:
show_test_game(env, rollout_stategy, q_3x3_strategy)

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   | x | 
---- ---- ---- -

---- ---- ---- -
| o |   | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
|   | x | x | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x | o

In [19]:
show_test_game(env, q_3x3_strategy, rollout_stategy)

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
| x |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
| x |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
|   | x | x | 
---- ---- ---- -
| x |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x | x | o | 
---- ---- ---- -

---- ---- ---- -
|   | o | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x | x | o | 
---- ---- ---- -

---- ---- ---- -
| x | o | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x | x

Против стратегии, обученной во второй части

In [20]:
class DuelingQNetwork(nn.Module):
    def __init__(self, num_filters=32, in_channels=2):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, num_filters, 3, padding=1)
        self.conv2 = nn.Conv2d(num_filters, num_filters, 3, padding=1)

        self.v_conv1 = nn.Conv2d(num_filters, 1, 1)
        self.v_conv2 = nn.Conv2d(1, 1, 3, padding=0)

        self.a_conv = nn.Conv2d(num_filters, 1, 1)

    def forward(self, x):
        batch_size = x.shape[0]
        x = F.leaky_relu(self.conv1(x))
        x = F.leaky_relu(self.conv2(x))
        
        v = F.leaky_relu(self.v_conv1(x))
        v = self.v_conv2(v)
        
        a = self.a_conv(x)
        a_mean = a.view(batch_size, -1).mean(-1).reshape(batch_size, 1, 1, 1)
        q = v + (a - a_mean)
        
        return q

In [21]:
dueling_model = DuelingQNetwork()

In [22]:
path = 'models/dueling_dqn_weights_3x3.pt'
dueling_model.load_state_dict(torch.load(path))
dueling_model.eval();

In [23]:
greedy_dueling_dqn_strategy = DQNStrategy(dueling_model, epsilon=0, n_cols=env.env.n_cols)

In [24]:
show_test_game(env, rollout_stategy, greedy_dueling_dqn_strategy)

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   | x | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   | x | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   | o |   | 
---- ---- ---- -

---- ---- ---- -
|   | x | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
| x | x |   | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
| x | x | o | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
| x | x | o | 
---- ---- ---- -
| x | o

In [25]:
show_test_game(env, greedy_dueling_dqn_strategy, rollout_stategy)

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
| x | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
| x | x | o | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
| x | x | o | 
---- ---- ---- -
|   |   | x | 
---- ---- ---- -

---- ---- ---- -
| o |   | o | 
---- ---- ---- -
| x | x | o | 
---- ---- ---- -
|   |   | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
| x | x | o | 
---- ---- ---- -
|   |   | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
| x | x | o | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
| x | x | o | 
---- ---- ---- -
| x | o

## Пункт 6
Реализуйте MCTS-поиск с этими rollouts для крестиков-ноликов на досках разного
размера, сравните полученные стратегии между собой и со стратегиями,
обученными в первых двух частях.

In [114]:
from importlib import reload

import mcts
reload(mcts)

from mcts import rollout, RolloutStrategy, MCTSNode, MCTSStrategy

In [115]:
mcts_strategy = MCTSStrategy(env,
                            base_strategy=RandomStrategy(),
                            num_selections=1000,
                            num_rollouts=200)

MCTS против случайной стратегии

In [116]:
show_test_game(env, mcts_strategy, random_strategy)

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x | x | 
---- ---- ---- -
|   |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x | x | 
---- ---- ---- -
| o |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
| x | x | x | 
---- ---- ---- -
| o |   | o | 
---- ---- ---- -

Крестики выиграли!


In [117]:
show_test_game(env, random_strategy, mcts_strategy)

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   |   | x | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | o |   | 
---- ---- ---- -
|   |   | x | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -
|   |   | x | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -
|   |   | x | 
---- ---- ---- -

---- ---- ---- -
| x |   | o | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -
|   |   | x | 
---- ---- ---- -

---- ---- ---- -
| x |   | o | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -
| o |   | x | 
---- ---- ---- -

Нолики выиграли!


MCTS против самого себя

In [118]:
show_test_game(env, mcts_strategy, mcts_strategy)

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x | x | 
---- ---- ---- -
|   |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
|   |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x |   | o | 
---- ---- ---- -

---- ---- ---- -
| x |   | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x |   | o | 
---- ---- ---- -

---- ---- ---- -
| x | o | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x |   | o | 
---- ---- ---- -

---- ---- ---- -
| x | o | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x | x

MCTS против роллаутов

In [119]:
show_test_game(env, mcts_strategy, rollout_stategy)

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x | x | 
---- ---- ---- -
|   |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x | x | 
---- ---- ---- -
| o |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
| x | x | x | 
---- ---- ---- -
| o |   | o | 
---- ---- ---- -

Крестики выиграли!


In [120]:
show_test_game(env, rollout_stategy, mcts_strategy)

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
| x |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
| x |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
|   | x | x | 
---- ---- ---- -
| x |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x | x | o | 
---- ---- ---- -

---- ---- ---- -
|   | o | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x | x | o | 
---- ---- ---- -

---- ---- ---- -
| x | o | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x | x

MCTS против стратегии, обученной в первой части

In [121]:
show_test_game(env, mcts_strategy, q_3x3_strategy)

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   | x | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   | x | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   | o |   | 
---- ---- ---- -

---- ---- ---- -
|   | x | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
| x | x |   | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
| x | x | o | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
| x | x | o | 
---- ---- ---- -
| x | o

In [122]:
show_test_game(env, q_3x3_strategy, mcts_strategy)

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x | x | 
---- ---- ---- -
|   |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
|   |   | o | 
---- ---- ---- -

---- ---- ---- -
|   | x |   | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
|   |   | o | 
---- ---- ---- -

---- ---- ---- -
|   | x |   | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
|   | o | o | 
---- ---- ---- -

---- ---- ---- -
|   | x |   | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x | o | o | 
---- ---- ---- -

---- ---- ---- -
|   | x | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x | o | o | 
---- ---- ---- -

---- ---- ---- -
| x | x | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x | o

Против стратегии, обученной во второй части

In [123]:
show_test_game(env, mcts_strategy, greedy_dueling_dqn_strategy)

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   | x | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   | x | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   | o |   | 
---- ---- ---- -

---- ---- ---- -
|   | x | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
| x | x |   | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
| x | x | o | 
---- ---- ---- -
|   | o | x | 
---- ---- ---- -

---- ---- ---- -
| o | x | o | 
---- ---- ---- -
| x | x | o | 
---- ---- ---- -
| x | o

In [124]:
show_test_game(env, greedy_dueling_dqn_strategy, mcts_strategy)

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   |   | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
|   |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   |   | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
| x |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
|   | x |   | 
---- ---- ---- -
| x |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
|   | x | x | 
---- ---- ---- -
| x |   | o | 
---- ---- ---- -

---- ---- ---- -
|   |   | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x |   | o | 
---- ---- ---- -

---- ---- ---- -
|   | x | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x |   | o | 
---- ---- ---- -

---- ---- ---- -
|   | x | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x | o | o | 
---- ---- ---- -

---- ---- ---- -
| x | x | o | 
---- ---- ---- -
| o | x | x | 
---- ---- ---- -
| x | o

## MCTS на доске 4x4

In [129]:
env = TicTacToeDQNWrapper(TicTacToe(4,4,4))

In [130]:
mcts_strategy_4x4 = MCTSStrategy(env,
                                base_strategy=RandomStrategy(),
                                num_selections=1000,
                                num_rollouts=200)

MCTS против случайной стратегии

In [131]:
show_test_game(env, mcts_strategy_4x4, random_strategy)

---- ---- ---- ---- -
| x |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
| x |   |   |   | 
---- ---- ---- ---- -
| o |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
| x |   |   |   | 
---- ---- ---- ---- -
| o | x |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
| x |   |   |   | 
---- ---- ---- ---- -
| o | x |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   | o | 
---- ---- ---- ---- -

---- ---- ---- ---- -
| x |   |   |   | 
---- ---- ---- ---- -
| o | x |   |   | 
---- ---- ---- ---- -
|   |   |   | x | 
---- ---- ---- ---- -
|   |   |   | o | 
---- ---- ---- ---- -

---- ---- ---- ---- -
| x |   |   |   | 
---- ---- ---- ---- -
| 

In [132]:
show_test_game(env, random_strategy, mcts_strategy_4x4)

---- ---- ---- ---- -
|   |   | x |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
|   |   | x |   | 
---- ---- ---- ---- -
|   |   | o |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
|   |   | x |   | 
---- ---- ---- ---- -
|   |   | o |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   | x | 
---- ---- ---- ---- -

---- ---- ---- ---- -
|   |   | x | o | 
---- ---- ---- ---- -
|   |   | o |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   | x | 
---- ---- ---- ---- -

---- ---- ---- ---- -
|   |   | x | o | 
---- ---- ---- ---- -
|   |   | o |   | 
---- ---- ---- ---- -
|   |   |   | x | 
---- ---- ---- ---- -
|   |   |   | x | 
---- ---- ---- ---- -

---- ---- ---- ---- -
|   |   | x | o | 
---- ---- ---- ---- -
| 

MCTS против роллаутов

In [134]:
rollout_stategy_4x4 = RolloutStrategy(env, n_iter=10000, n_jobs=16, base_strategy=RandomStrategy())

In [135]:
show_test_game(env, mcts_strategy_4x4, rollout_stategy_4x4)

---- ---- ---- ---- -
| x |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
| x |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   | o |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
| x |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   | x | o |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
| x |   |   |   | 
---- ---- ---- ---- -
|   |   | o |   | 
---- ---- ---- ---- -
|   | x | o |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
| x |   |   |   | 
---- ---- ---- ---- -
|   |   | o |   | 
---- ---- ---- ---- -
|   | x | o |   | 
---- ---- ---- ---- -
|   |   | x |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
| x |   |   |   | 
---- ---- ---- ---- -
| 

In [136]:
show_test_game(env, rollout_stategy_4x4, mcts_strategy_4x4)

---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   | x |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   | o |   |   | 
---- ---- ---- ---- -
|   | x |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   | o |   |   | 
---- ---- ---- ---- -
|   | x | x |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   | o |   |   | 
---- ---- ---- ---- -
| o | x | x |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   | o | x |   | 
---- ---- ---- ---- -
| o | x | x |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
|   |   |   | o | 
---- ---- ---- ---- -
| 

MCTS против стратегии, обученной в первой части

In [137]:
self_play_q_4x4 = load('models/self_play_q_4x4.pkl')

In [138]:
q_4x4_strategy = EpsilonGreedyStrategy(0, self_play_q_4x4)

In [140]:
show_test_game(env, mcts_strategy_4x4, q_4x4_strategy)

---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   | x |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   | o |   |   | 
---- ---- ---- ---- -
|   | x |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   | o |   |   | 
---- ---- ---- ---- -
|   | x | x |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
| o |   |   |   | 
---- ---- ---- ---- -
|   | o |   |   | 
---- ---- ---- ---- -
|   | x | x |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
| o |   |   | x | 
---- ---- ---- ---- -
|   | o |   |   | 
---- ---- ---- ---- -
|   | x | x |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
| o |   |   | x | 
---- ---- ---- ---- -
| 

In [141]:
show_test_game(env, q_4x4_strategy, mcts_strategy_4x4)

---- ---- ---- ---- -
|   |   |   | x | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
| o |   |   | x | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -

---- ---- ---- ---- -
| o |   |   | x | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   | x | 
---- ---- ---- ---- -

---- ---- ---- ---- -
| o |   |   | x | 
---- ---- ---- ---- -
|   |   |   |   | 
---- ---- ---- ---- -
|   |   |   | o | 
---- ---- ---- ---- -
|   |   |   | x | 
---- ---- ---- ---- -

---- ---- ---- ---- -
| o |   |   | x | 
---- ---- ---- ---- -
|   |   | x |   | 
---- ---- ---- ---- -
|   |   |   | o | 
---- ---- ---- ---- -
|   |   |   | x | 
---- ---- ---- ---- -

---- ---- ---- ---- -
| o |   |   | x | 
---- ---- ---- ---- -
| 